In [25]:
!pip install rpy2 

In [1]:
rebuild = False

In [2]:
import os
import pandas as pd
import time
from nltk.util import skipgrams
from nltk.lm import NgramCounter

In [3]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from rpy2.robjects import StrVector, IntVector
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
r_libs = '/home/krzys/R/x86_64-pc-linux-gnu-library/4.1'# R libs
base = importr('base')
print(base._libPaths())
utils.install_packages("wordspace")
wordspace = importr("wordspace", lib_loc=r_libs)

R[write to console]: Instalowanie pakietu w ‘/home/krzys/R/x86_64-pc-linux-gnu-library/4.1’
(ponieważ ‘lib’ nie jest określony)



[1] "/home/krzys/R/x86_64-pc-linux-gnu-library/4.1"
[2] "/usr/local/lib/R/site-library"                
[3] "/usr/lib/R/site-library"                      
[4] "/usr/lib/R/library"                           



R[write to console]: próbowanie adresu URL 'https://cloud.r-project.org/src/contrib/wordspace_0.2-7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1840058 bytes (1.8 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R

g++ -std=gnu++14 -I"/usr/share/R/include" -DNDEBUG  -I'/home/krzys/R/x86_64-pc-linux-gnu-library/4.1/Rcpp/include'   -fopenmp -fpic  -g -O2 -ffile-prefix-map=/build/r-base-4A2Reg/r-base-4.1.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c RcppExports.cpp -o RcppExports.o
g++ -std=gnu++14 -I"/usr/share/R/include" -DNDEBUG  -I'/home/krzys/R/x86_64-pc-linux-gnu-library/4.1/Rcpp/include'   -fopenmp -fpic  -g -O2 -ffile-prefix-map=/build/r-base-4A2Reg/r-base-4.1.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c wordspace.cpp -o wordspace.o
g++ -std=gnu++14 -shared -L/usr/lib/R/lib -Wl,-Bsymbolic-functions -flto=auto -ffat-lto-objects -flto=auto -Wl,-z,relro -o wordspace.so RcppExports.o wordspace.o -fopenmp -L/usr/lib/R/lib -lR


installing to /home/krzys/R/x86_64-pc-linux-gnu-library/4.1/00LOCK-wordspace/00new/wordspace/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (wordspace)
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/RtmpEoi53X/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [4]:
# define terms we're interested in
socio_political_terms = ["civitas", "consilium", "consul", "dux", "gens", "hostis", "imperator", "jus", "labor", "natio", "nobilitas", "pontifex", "pontificium", "populus", "potestas", "regnum", "senatus", "sodes", "urbs"]

In [16]:
# prepare metadata
dir_in = os.path.join("/home/krzys/Kod/streamlit/voces/data/corpora/latinise_IT_lemmas/")
dir_in
files = os.listdir(os.path.join(dir_in))
files = [f for f in files[:] if "IT" in f]
metadata_df = pd.read_csv(os.path.join(dir_in, 'latinise_metadata.csv'), sep = ",")
metadata_df = metadata_df[metadata_df['id'].str.startswith("IT")]
metadata_df.head()

# split the corpus into subcorpora
first_date = min(metadata_df.date)
last_date = max(metadata_df.date)
print(first_date)
print(last_date)
size_interval = 500
n_intervals = round((last_date-first_date)/size_interval)
n_intervals
intervals = [None]*(n_intervals+1)
for t in range(n_intervals+1):
    #print(t)
    if t == 0:
        intervals[t] = first_date
    else:
        intervals[t] = intervals[t-1]+size_interval
    #print(intervals[t])
    
print(intervals)

metadata_df['time_interval'] = ""
for t in range(len(intervals)-1):
    print(t)
    print(range(intervals[t],intervals[t+1]))
    metadata_df_t = metadata_df.loc[metadata_df['date'].isin(range(intervals[t],intervals[t+1]))]
    print(metadata_df_t.date)
    metadata_df.loc[metadata_df['date'].isin(range(intervals[t],intervals[t+1])),'time_interval'] = intervals[t]
metadata_df

def convert_dates(sign, date0):

    if sign == "0":
        if date0 == 0:
            final_date = "+0000"
        elif date0 < 100:
            final_date = "+" + "00" + str(date0)
            #print("1-final_date", final_date)
        elif date0 < 1000:
            final_date = "+" + "0" + str(date0)
            #print("2-final_date", final_date)
        else:
            final_date = "+" + str(date0)
            #print("3-final_date", final_date)
    else:
        if date0 == 0:
            final_date = "+0000"
        elif date0 < 100:
            final_date = str(sign) + "00" + str(date0)
            #print("1-final_date", final_date)
        elif date0 < 1000:
            final_date = str(sign) + "0" + str(date0)
            #print("2-final_date", final_date)
        else:
            final_date = str(sign) + str(date0)
            #print("3-final_date", final_date)

    if final_date.startswith("+"):
        final_date = final_date.replace("+", "")
    return final_date

-450
2005
[-450, 50, 550, 1050, 1550, 2050]
0
range(-450, 50)
19      -9
34     -49
38      49
39     -45
42     -49
      ... 
635   -149
638   -107
642    -37
643    -37
649   -229
Name: date, Length: 91, dtype: int64
1
range(50, 550)
18     382
20     524
23     399
24     391
37     158
      ... 
683    116
684    116
685    116
686    116
687    533
Name: date, Length: 236, dtype: int64
2
range(550, 1050)
102    800
104    800
105    800
106    800
107    800
      ... 
601    833
609    598
628    950
634    550
636    550
Name: date, Length: 66, dtype: int64
3
range(1050, 1550)
21     1254
22     1254
25     1224
26     1150
28     1517
       ... 
624    1067
625    1110
626    1110
644    1501
646    1483
Name: date, Length: 178, dtype: int64
4
range(1550, 2050)
27     2000
33     1998
35     2001
36     1992
220    1998
       ... 
641    1685
647    1550
650    1555
651    1684
652    1684
Name: date, Length: 99, dtype: int64


In [17]:
# prepare the corpus
punctuation = ['.', ',', '...', ';', ':', '?']

corpus = list()
corpus_filenames = list()
files_corpus = metadata_df
for index, df_line in files_corpus.iterrows():
    #print("line:",df_line['id'], df_line['time_interval'])
    sign = "+"
    #print(df_line['date'])
    if df_line['date'] < 0:
        sign = "-"
    #print("date:", convert_dates(sign, abs(df_line['date'])))
    file_name = 'lat_'+str(convert_dates(sign, abs(df_line['date'])))+"_"+str(df_line['id'])+'.txt'
    #KN: missing files
    if os.path.isfile(os.path.join(dir_in, file_name)):
        file = open(os.path.join(dir_in, file_name), 'r')
        sentences_this_file = list()
        while True:
            line = file.readline().strip()
            if line != "":
                corpus.append([token.lower() for token in line.split(" ") if token not in punctuation]) #KN: lemmas to lowcase
                corpus_filenames.append(file_name) #KN: track the file name for the sentence
            # if line is empty end of file is reached
            if not line:
                break
        file.close()
        corpus.append(sentences_this_file)
    else:
        print(os.path.join(dir_in, file_name), " doesn't exist")

In [20]:
if rebuild == True:
    #Let's compute cooccurrence counts in the corpus.
    window=2
    skip=0 # defines distance if skipgram
    ngrams = [ skipgrams(sent, window, skip) for sent in corpus ]
    ngram_counts = NgramCounter(ngrams)

In [21]:
# get triples (target, feature, freq) from the ngram_counts
if rebuild == True:
    coocs = []
    for node, freqs in ngram_counts[window].items(): # dict_items([(('ars',), FreqDist({'et': 149, 'sum': 129, ...}}))])
        #print(word)
        for cooc, freq  in freqs.items():
            triple = (node[0], cooc, freq) # node term, cooc, freq
            coocs.append(triple)
    print(coocs[0:10])

[('ars', 'amatorius', 2), ('ars', 'populus', 1), ('ars', 'cieo', 1), ('ars', 'levis', 1), ('ars', 'rego', 2), ('ars', 'ferus', 2), ('ars', 'nec', 9), ('ars', 'sum', 129), ('ars', 'careo', 4), ('ars', 'excutio', 1)]


In [22]:
# convert cooc counts to matrix
coocs_df = pd.DataFrame(coocs, columns=["target", "feature", "score"])
coocs_df.head()

,target,feature,score
0,ars,amatorius,2
1,ars,populus,1
2,ars,cieo,1
3,ars,levis,1
4,ars,rego,2


In [23]:
if rebuild == True:
    # create dsm matrix
    VObj = wordspace.dsm(target=StrVector(coocs_df["target"]),
                         feature=StrVector(coocs_df["feature"]),
                         score=IntVector(coocs_df["score"]),
                         raw_freq=True) 
    print(VObj)

Distributional Semantic Model with 192519 rows x 196956 columns
* raw co-occurrence matrix M available
  - sparse matrix with 2779.5k / 37.9G nonzero entries (fill rate = 0.01%)
  - in canonical format
  - known to be non-negative
  - sample size of underlying corpus: 7832.7k tokens



In [24]:
if rebuild == True:
    #default options
    config = dict(score="simple-ll", transform="log", normalize=True, method="euclidean", reduce_method="svd", reduce_n=300)
    print(config)
    VObj_weighted = wordspace.dsm_score(VObj, score=config["score"], transform=config["transform"], normalize=config["normalize"], method=config["method"])
    print(VObj_weighted)

{'score': 'simple-ll', 'transform': 'log', 'normalize': True, 'method': 'euclidean', 'reduce_method': 'svd', 'reduce_n': 300}
Distributional Semantic Model with 192519 rows x 196956 columns
* raw co-occurrence matrix M available
  - sparse matrix with 2779.5k / 37.9G nonzero entries (fill rate = 0.01%)
  - in canonical format
  - known to be non-negative
  - sample size of underlying corpus: 7832.7k tokens
* scored matrix S available
  - sparse matrix with 2500.3k / 37.9G nonzero entries (fill rate = 0.01%)
  - in canonical format
  - known to be non-negative



In [25]:
if rebuild == True:
    VObj_weighted_reduced = wordspace.dsm_projection(VObj_weighted, method=config["reduce_method"], n=config["reduce_n"])
    print(VObj_weighted_reduced[0]) # takes around 5 mins

0.03863485012165868


In [26]:
if rebuild == True:
    # save matrices to disk
    matrices = [VObj_weighted, VObj_weighted_reduced]
    import pickle
    f = open('dsm_matrices.model', 'wb')
    pickle.dump(matrices,f)

if rebuild == False:
    import pickle
    f = open('dsm_matrices.model', 'rb')
    matrices = pickle.load(f)
    VObj_weighted = matrices[0]
    VObj_weighted_reduced = matrices[1]

In [27]:
# get similar from unreduced space
[ print(term, "\n", wordspace.nearest_neighbours(VObj_weighted, term, n=10)) for term in socio_political_terms ]
# get similar from reduced space
[ print(term, "\n", wordspace.nearest_neighbours(VObj_weighted_reduced, term, n=10)) for term in socio_political_terms ]

civitas 
       urbs beneventum    castrum      regio  exercitus  provincia      civis 
  78.55179   82.71630   82.83374   82.92972   83.02814   83.31888   83.38774 
    italia       roma      domus 
  83.53758   83.58676   83.61037 

consilium 
      ratio   amicus#2       cura     animus   voluntas   iudicium      fides 
  82.73884   83.20646   83.21133   83.28279   83.33736   83.62575   83.64608 
  negotium  hebdomade diligentia 
  83.66563   83.68386   83.78745 

consul 
  praetor dictator   censor hispania  collega  curulis hannibal   scipio 
74.73342 76.18597 79.04988 79.58169 80.10038 80.61333 80.76245 80.87737 
extemplo  sicilia 
80.88246 81.03433 

dux 
 exercitus       rex     miles  princeps    consul  proelium     comes  auxilium 
 82.44744  83.24561  83.39639  83.47414  83.78076  83.84675  83.87690  83.94952 
  legatus   classis 
 84.13482  84.26266 

gens 
        rex      natio      regio    populus  exercitus  provincia        tam 
  83.25631   83.38656   83.63510   83.

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [28]:
# dictionary that maps a time interval with the list of sentences of texts in that time interval"
time2corpus = dict()

# I loop over all time intervals:
for t in range(n_intervals+1):
    files_corpus_t = metadata_df.loc[metadata_df['time_interval'] == intervals[t]]
    #print("1:",files_corpus_t, type(files_corpus_t))
    corpus_t = list()
    for index, df_line in files_corpus_t.iterrows():
        #print("line:",df_line['id'], df_line['time_interval'])
        sign = "+"
        #print(df_line['date'])
        if df_line['date'] < 0:
            sign = "-"
        #print("date:", convert_dates(sign, abs(df_line['date'])))
        file_name = 'lat_'+str(convert_dates(sign, abs(df_line['date'])))+"_"+str(df_line['id'])+'.txt'
        #print("3:",file_name)
        #KN: missing files
        if os.path.isfile(os.path.join(dir_in, file_name)):
            file = open(os.path.join(dir_in, file_name), 'r')
            sentences_this_file = list()
            while True:
                line = file.readline().strip()
                if line != "":
                    #sentences_this_file.append(line.split(" "))
                    #sentences_this_file.append([token for token in line.split(" ") if token not in punctuation])
                    corpus_t.append([token.lower() for token in line.split(" ") if token not in punctuation]) #KN: tolower
                # if line is empty end of file is reached
                if not line:
                    break
            file.close()
        #corpus_t.append(sentences_this_file)
    #corpus_t1
    #print(len(corpus_t1[0]))
    time2corpus[t] = corpus_t

In [29]:
time2corpus[4][0:2]

[['dominus',
  'iesus',
  'antequam',
  'in',
  'coelum',
  'ascendo',
  'suus',
  'discipulus',
  'mandatum',
  'confero',
  'nuntio',
  'evangelium',
  'cunctus',
  'homo',
  'omnesque',
  'populus',
  'baptizo',
  'eo',
  'in',
  'mundum',
  'universus',
  'praedico#1',
  'evangelium',
  'omnis',
  'creatura'],
 ['qui',
  'credo',
  'et',
  'baptizo',
  'sum',
  'salvus',
  'sum',
  'quis#2',
  'verus',
  'non',
  'credo',
  'condemno',
  '(',
  'mc',
  '-',
  ')',
  'data',
  'sum',
  'ego',
  'omnis',
  'potestas',
  'in',
  'caelum',
  'et',
  'in',
  'terra']]

In [30]:
if rebuild == True:
    window=2
    skip=0 # defines distance if skipgram
    # DSM options
    config = dict(score="simple-ll", transform="log", normalize=True, method="euclidean", reduce_method="svd", reduce_n=300)
    models = dict.fromkeys(range(len(time2corpus)))
    #models

In [31]:
if rebuild == True:
    start = time.time()
    for t in range(len(time2corpus)):
        if len(time2corpus[t])>0:
            print("Building model for t = ", t, "\n")

            print("Retrieving coocurrence counts for t = ", t, "\n")
            #prepare coocs
            ngrams = [ skipgrams(sent, window, skip) for sent in time2corpus[t] ]
            ngram_counts = NgramCounter(ngrams)

            print("Retrieving (target, feature, freq) from the ngram_counts for t = ", t, "\n")
            # get triples (target, feature, freq) from the ngram_counts
            coocs = []
            for node, freqs in ngram_counts[window].items(): # dict_items([(('ars',), FreqDist({'et': 149, 'sum': 129, ...}}))])
                for cooc, freq  in freqs.items():
                    triple = (node[0], cooc, freq) # node term, cooc, freq
                    coocs.append(triple)

            coocs_df = pd.DataFrame(coocs, columns=["target", "feature", "score"])
            #coocs_df.head()

            print("Building DSM matrices for t = ", t, "\n")

            # create dms matrix
            VObj = wordspace.dsm(target=StrVector(coocs_df["target"]),
                                 feature=StrVector(coocs_df["feature"]),
                                 score=IntVector(coocs_df["score"]),
                                 raw_freq=True) 
            VObj_weighted = wordspace.dsm_score(VObj, score=config["score"], transform=config["transform"], normalize=config["normalize"], method=config["method"])
            VObj_weighted_reduced = wordspace.dsm_projection(VObj_weighted, method=config["reduce_method"], n=config["reduce_n"])

            print("Saving models for t = ", t, "\n")
            models[t] = (VObj_weighted, VObj_weighted_reduced)

    end = time.time()
    print("It has taken", round(end - start), "seconds, or ", round((end - start)/60), "minutes")

Building model for t =  0 

Retrieving coocurrence counts for t =  0 

Retrieving (target, feature, freq) from the ngram_counts for t =  0 

Building DSM matrices for t =  0 

Saving models for t =  0 

Building model for t =  1 

Retrieving coocurrence counts for t =  1 

Retrieving (target, feature, freq) from the ngram_counts for t =  1 

Building DSM matrices for t =  1 

Saving models for t =  1 

Building model for t =  2 

Retrieving coocurrence counts for t =  2 

Retrieving (target, feature, freq) from the ngram_counts for t =  2 

Building DSM matrices for t =  2 

Saving models for t =  2 

Building model for t =  3 

Retrieving coocurrence counts for t =  3 

Retrieving (target, feature, freq) from the ngram_counts for t =  3 

Building DSM matrices for t =  3 

Saving models for t =  3 

Building model for t =  4 

Retrieving coocurrence counts for t =  4 

Retrieving (target, feature, freq) from the ngram_counts for t =  4 

Building DSM matrices for t =  4 

Saving model

In [32]:
if rebuild == True:
    # save diachronic models to disk
    import pickle
    with open('dsm_matrices_time.model', 'wb') as f:
        pickle.dump(models,f)

In [5]:
if rebuild == False:
    #load models from disk
    import pickle
    with open('dsm_matrices_time.model', 'rb') as f:
        models = pickle.load(f)

In [6]:
# get similarity counts
for t, model in models.items():
    if model is not None:
        print("Similarities for t =", t)
        print("Unreduced matrix for t =", t)    
        [ print(term, "\n", wordspace.nearest_neighbours(model[0], term, n=10) 
                if term in list(models[t][0].rx2('rows').rx2("term")) else None)
         for term in socio_political_terms ]
        print("Reduced matrix for t =", t)
        [ print(term, "\n", wordspace.nearest_neighbours(model[1], term, n=10)  
                if term in list(models[t][0].rx2('rows').rx2("term")) else None) 
         for term in socio_political_terms ]

Similarities for t = 0
Unreduced matrix for t = 0
civitas 
        finibusque          publicus     exercituumque       sermonesque 
         80.51078          80.66990          81.10962          81.13620 
            fetus       conservatio          iuturnae           luceres 
         81.55200          82.01738          82.29417          82.29417 
           fasque immortalitatemque 
         82.56356          82.57006 

consilium 
  caedisque   arrenius    praemia     emunio liberumque    toronen      fides 
  81.68704   81.68704   81.68704   82.21108   82.62868   82.63089   82.70372 
    animus mulciberis  siliculam 
  82.70840   82.74664   82.74664 

consul 
   praetor  dictator      creo provincia      roma  hispania    censor     annus 
 74.67951  77.49030  78.77009  79.09332  79.64992  79.81656  80.12949  80.36045 
  curulis   urbanus 
 80.42726  80.57770 

dux 
     inferentem      imperator        fautore   libertateque consololatione 
      81.74230       82.38217       82.5

## Aligning vector spaces

We're extracting vectors from the R objects created with wordspace and saving them as dfs indexed with lemmas (the column names are svd_1..svd_dimensions).

In [10]:
if rebuild == True:
    import pandas as pd
    import rpy2.robjects as ro
    from rpy2.robjects.packages import importr
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.conversion import localconverter

    models_dfs = {}
    for period in models.keys():
        if models[period] is None:
            print(period, " is None")
            continue
        else:
            print(period)
            models_dfs.setdefault(period, pd.DataFrame())
            with localconverter(ro.default_converter + pandas2ri.converter):
                model_arr = ro.conversion.rpy2py(models[period][1])        
            model_df = pd.DataFrame(model_arr)
            model_df.index = models[period][1].names[0]
            model_df.columns = models[period][1].names[1]
            models_dfs[period] = model_df
        
    import pickle
    with open('dsm_matrices_time_dfs.pickle', 'wb') as f:
        pickle.dump(models_dfs,f)

if rebuild == False:
    import pickle
    with open('dsm_matrices_time_dfs.pickle', 'rb') as f:
        models_dfs = pickle.load(f)

Let's inspect the data.

In [11]:
models_dfs[0].head()

,svd1,svd2,svd3,svd4,svd5,svd6,svd7,svd8,svd9,svd10,...,svd291,svd292,svd293,svd294,svd295,svd296,svd297,svd298,svd299,svd300
ars,0.095061,-0.082544,0.008310,-0.045631,0.000817,-0.042844,0.042958,-0.046266,0.063054,0.004916,...,0.001986,-0.012368,-0.003418,-0.035642,0.002080,-0.001460,0.001799,-0.010752,0.002475,0.008507
amatorius,0.000163,-0.000185,0.000071,-0.000167,0.000047,-0.000403,0.000285,-0.000335,0.000245,-0.000093,...,0.000472,-0.000408,0.001617,-0.000630,0.002456,-0.001729,-0.000765,-0.000872,0.000771,-0.000881
siquis,0.033042,-0.104033,0.170928,-0.209175,-0.047206,-0.001350,0.004896,-0.009733,0.010704,-0.007433,...,0.005240,-0.010728,-0.013535,0.007165,-0.018178,0.010318,-0.002000,-0.000389,-0.008294,0.007803
in,0.014464,-0.014608,0.020222,-0.024395,0.075065,-0.031054,0.027553,-0.023013,0.014513,-0.009114,...,0.025527,-0.030701,0.013832,0.003147,0.044565,-0.000151,0.004925,-0.028451,-0.007531,0.001007
hic,0.013815,-0.014330,0.014638,-0.016489,0.010713,-0.039562,0.031750,-0.027423,0.015492,-0.009739,...,-0.004887,0.000819,0.042853,-0.013527,0.000618,-0.012543,-0.001751,0.010299,0.018531,-0.040335


In [12]:
def intersection_align_wordspace(m1, m2, words=None): # modified version of the gensim-based code
    """
    Intersect two wordspace count models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.index)
    vocab_m2 = set(m2.index)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (common_vocab, m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    #common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True) # KN: no sorting needed
    # print(len(common_vocab))

    # Then for each model...
    m1 = m1.loc[common_vocab]
    m2 = m2.loc[common_vocab]
    
    return (common_vocab, m1, m2)

Let's test matrix alignment. As we don't have time now to modify `wordspace` `R` objects we'll be working with numpy arrays and convert them to lemma-indexed pandas dfs.

In [ ]:
common, m1, m2 = intersection_align_wordspace(models_dfs[0], models_dfs[1])
from scipy.linalg import orthogonal_procrustes # https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.orthogonal_procrustes.html#rb53e47558d72-1
import numpy as np
test = orthogonal_procrustes(np.array(m1).T, np.array(m2).T)
test[0].shape # Solution. The test[1] stores the sum of singular values
test_procrustes = pd.DataFrame(test[0])

# Finished here

In [ ]:
def smart_procrustes_align_wordspace(base_embed, other_embed, words=None):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # make sure vocabulary and indices are aligned
    common_vocab, in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

    # re-filling the normed vectors: the following two lines were added following Japleen Gulati, following amacanovic's comments n the discussion below this page: https://gist.github.com/zhicongchen/9e23d5c3f1e5b1293b16133485cd17d8

    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)
    
    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    return other_embed


For reference: the structure of wordspace objects and how to access them.

In [ ]:
#models[0][0].rx2('rows').rx2("term")

#print("Let's inspect a model: ", [ print(i) for i in models[0] ] )
#print("Let's inspect a model: ", [ len(i) for i in models[1] ] )
#print(models[1][0])
# weighted = models[0][0]
# weighted reduced = models[0][1]

#print(models[0][0]) #weighted matrix

#print(models[0][0][0]) #sparse Matrix of class "dgCMatrix"
#same as: models[0][0].rx2('M') RAW!
#print(models[0][0][1]) #term - nnzero - f
#same as: models[0][0].rx2('rows')
#print(models[0][0][2]) #term - nnzero - f
#same as: models[0][0].rx2('cols')
#print(models[0][0][3]) # $N, $locked
#same as: models[0][0].rx2('globals
#print(models[0][0][4]) # sparse Matrix of class "dgCMatrix"
#same as: models[0][0].rx2('S') WEIGHTED!

#print(models[0][1]) # reduced matrix
#print(len(models[0][1])) # reduced matrix length

#print(list(models[0][0].rx2('cols').rx2("term"))[0:10])
#list(models[0][0].rx2('cols')[0:10])
#print(list(models[0][0].rx2('rows').rx2("term"))[0:10])
#list(models[0][0].rx2('rows')[0:10])

#print(list(models[0].rx2('M')))